In [2]:
import pandas as pd
import numpy as np
import warnings
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.python.keras import callbacks
from keras import backend as K
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from keras.layers import Dense, Dropout
from sklearn.model_selection import KFold
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping

warnings.filterwarnings(action='ignore')
tf.config.set_visible_devices([], 'GPU')

In [4]:
df = pd.read_csv("data/modified_train_0417.csv")
x_train = df.drop(columns=['loan_status', 'addr_state', 'loan_amnt_intallment_rate'])
y_train = df['loan_status']

In [5]:
nan_indices = np.isnan(x_train).any(axis=1)
x_train = x_train[~nan_indices]
y_train = y_train[~nan_indices]

In [6]:
x_train.shape

(1091767, 93)

In [7]:
# minmax scaler 찾아보기
scaler = StandardScaler()
X_scaled = scaler.fit_transform(x_train)

# PCA로 차원 축소
pca = PCA(n_components=90)
X_pca = pca.fit_transform(X_scaled)
x_train = X_pca.astype('float32')
# stratify로 학습에 용이하게 비율 유지
x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)

In [8]:
y_train=to_categorical(y_train, 2).astype(int)
y_validation=to_categorical(y_validation, 2).astype(int)
# from sklearn.preprocessing import OneHotEncoder
# encoder = OneHotEncoder()
# y_train = y_train.values
# y_train = encoder.fit_transform(y_train.reshape(-1, 1))

In [9]:
def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())    
    f1_val = 2 * (precision * recall) / (precision + recall + K.epsilon())
    return f1_val

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

In [42]:
def build_model(optimizer):
    # Define and compile the model
    model = keras.Sequential([
        Dense(45, input_dim=90, activation='relu'),
        Dropout(0.2),
        Dense(45, activation='relu'),
        Dropout(0.2),
        Dense(45, activation='relu'),
        Dropout(0.2),
        Dense(10),
        Dense(2, activation='softmax')
    ])
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
    return model
skfold = KFold(random_state=30,
           n_splits=5,
           shuffle=True
          )
model = KerasClassifier(build_fn = build_model)
parameters = {'batch_size': [1000,
                             5000,
                             10000],
              'epochs': [20,
                         45,
                         60,
                         100],
              'optimizer': ['adam', 
                            'SGD'
                            ]}
grid_search = GridSearchCV(estimator = model,
                           param_grid = parameters,
                           cv = skfold)
# early_stopping = EarlyStopping(monitor='val_loss',min_delta=0.01)
grid_search.fit(x_train, y_train)
print("Best params:", grid_search.best_params_)
print("Best average accuracy: {}".format(grid_search.best_score_))

Epoch 1/20
699/699 [==============================] - 2s 3ms/step - loss: 0.4791 - accuracy: 0.7927
Epoch 2/20
699/699 [==============================] - 2s 3ms/step - loss: 0.4616 - accuracy: 0.7980
Epoch 3/20
699/699 [==============================] - 2s 3ms/step - loss: 0.4589 - accuracy: 0.7986
Epoch 4/20
699/699 [==============================] - 2s 3ms/step - loss: 0.4578 - accuracy: 0.7992
Epoch 5/20
699/699 [==============================] - 2s 3ms/step - loss: 0.4570 - accuracy: 0.7993
Epoch 6/20
699/699 [==============================] - 2s 3ms/step - loss: 0.4563 - accuracy: 0.7994
Epoch 7/20
699/699 [==============================] - 2s 3ms/step - loss: 0.4559 - accuracy: 0.7997
Epoch 8/20
699/699 [==============================] - 2s 3ms/step - loss: 0.4557 - accuracy: 0.7997
Epoch 9/20
699/699 [==============================] - 2s 3ms/step - loss: 0.4550 - accuracy: 0.8000
Epoch 10/20
699/699 [==============================] - 2s 3ms/step - loss: 0.4547 - accuracy: 0.8000

In [43]:
result = grid_search.cv_results_

{'mean_fit_time': array([ 45.42095404,  47.89461579, 104.45527601,  97.54565234,
        149.18702364, 140.41665769, 247.88077388, 140.99477043,
         19.06272731,  18.97233076,  43.81525879,  43.86375208,
         60.45980515,  60.96489749, 106.32526121, 101.73821311,
         18.62472057,  18.23737502,  41.62129273,  41.66838169,
         54.65958519,  57.03063121,  92.40877023,  95.03137202]),
 'std_fit_time': array([ 4.18659111,  4.7309525 ,  1.78704312,  0.94590561,  3.2688449 ,
         3.31714687, 47.99313522,  6.83385589,  0.17794671,  0.29221182,
         1.35748998,  2.25478417,  2.81170764,  5.39197645,  5.52072732,
         2.64836496,  0.95918566,  0.34961127,  1.1002516 ,  0.55731421,
         0.5650627 ,  1.46955747,  1.76527115,  1.66301756]),
 'mean_score_time': array([0.28272729, 0.35230808, 0.31787553, 0.30284367, 0.33635268,
        0.29905705, 0.34909291, 0.22709961, 0.19568734, 0.19638619,
        0.18784876, 0.23205476, 0.2659811 , 0.24409332, 0.22987838,
    

## 최적 모델(without EarlyStopping)
- avg accuracy: 0.8004517912864685
- avg F1-score: 0.8004525303840637
- avg recall: 0.8004526019096374
- avg precision: 0.8004526019096374

In [44]:
# 최고의 파라미터 : {'batch_size': 10000, 'epochs': 100, 'optimizer': 'adam'}
valid_accs, valid_f1s, valid_recalls, valid_precisions = [], [], [], []

# layers
model = keras.Sequential([
    Dense(45, input_dim=90, activation='relu'),
    Dropout(0.2),
    Dense(45, activation='relu'),
    Dropout(0.2),
    Dense(45, activation='relu'),
    Dropout(0.2),
    Dense(10),
    Dense(2, activation='softmax')
])
# optimizer
optimizer = Adam(learning_rate=0.01)
# compile
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy", f1_score, recall, precision])
kf = KFold(n_splits=5, shuffle=True, random_state=30)
for train_index, val_index in kf.split(x_train, y_train):
    X_train_fold, X_val_fold = x_train[train_index], x_train[val_index]
    Y_train_fold, Y_val_fold = y_train[train_index], y_train[val_index]

    # Train the model
    model.fit(X_train_fold, Y_train_fold, 
              batch_size=10000, 
              epochs=100, 
              verbose=1, 
              #callbacks=[EarlyStopping(min_delta=0.001, monitor="loss")]
              )

    # Evaluate the model on validation data
    valid_loss, valid_acc, valid_f1, valid_recall, valid_precision = model.evaluate(X_val_fold, Y_val_fold)
    valid_accs.append(valid_acc)
    valid_f1s.append(valid_f1)
    valid_recalls.append(valid_recall)
    valid_precisions.append(valid_precision)
    print("===================================")
    print("Validation accuracy:", valid_acc)
    print("Validation F1-score:", valid_f1)
    print("Validation recall:", valid_recall)
    print("Validation precision:", valid_precision)
print("###################################")
print("avg Validation accuracy:", np.mean(valid_accs))
print("avg Validation F1-score:", np.mean(valid_f1s))
print("avg Validation recall:", np.mean(valid_recalls))
print("avg Validation precision:", np.mean(valid_precisions))


Epoch 1/100
70/70 [==============================] - 2s 9ms/step - loss: 0.4766 - accuracy: 0.7947 - f1_score: 0.7947 - recall: 0.7947 - precision: 0.7947
Epoch 2/100
70/70 [==============================] - 1s 9ms/step - loss: 0.4605 - accuracy: 0.7979 - f1_score: 0.7979 - recall: 0.7979 - precision: 0.7979
Epoch 3/100
70/70 [==============================] - 1s 8ms/step - loss: 0.4588 - accuracy: 0.7988 - f1_score: 0.7988 - recall: 0.7988 - precision: 0.7988
Epoch 4/100
70/70 [==============================] - 1s 8ms/step - loss: 0.4577 - accuracy: 0.7991 - f1_score: 0.7991 - recall: 0.7991 - precision: 0.7991
Epoch 5/100
70/70 [==============================] - 1s 8ms/step - loss: 0.4567 - accuracy: 0.7994 - f1_score: 0.7994 - recall: 0.7994 - precision: 0.7994
Epoch 6/100
70/70 [==============================] - 1s 8ms/step - loss: 0.4561 - accuracy: 0.7993 - f1_score: 0.7993 - recall: 0.7993 - precision: 0.7993
Epoch 7/100
70/70 [==============================] - 1s 10ms/step - lo

## 최적 model(with EarlyStopping)
### earlystopping: min_delta was 0.001
- avg accuracy: 0.7999342799186706
- avg F1-score: 0.7999348163604736
- avg recall: 0.7999348998069763
- avg precision: 0.7999348998069763

### earlystopping: min_delta was 0.0001
- avg accuracy: 0.80015869140625
- avg F1-score: 0.8001592040061951
- avg recall: 0.8001593112945556
- avg precision: 0.8001593112945556

In [47]:
# 최고의 파라미터 : {'batch_size': 10000, 'epochs': 100, 'optimizer': 'adam'}
valid_accs, valid_f1s, valid_recalls, valid_precisions = [], [], [], []

# layers
model = keras.Sequential([
    Dense(45, input_dim=90, activation='relu'),
    Dropout(0.2),
    Dense(45, activation='relu'),
    Dropout(0.2),
    Dense(45, activation='relu'),
    Dropout(0.2),
    Dense(10),
    Dense(2, activation='softmax')
])
# optimizer
optimizer = Adam(learning_rate=0.01)
# compile
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy", f1_score, recall, precision])
kf = KFold(n_splits=5, shuffle=True, random_state=30)
for train_index, val_index in kf.split(x_train, y_train):
    X_train_fold, X_val_fold = x_train[train_index], x_train[val_index]
    Y_train_fold, Y_val_fold = y_train[train_index], y_train[val_index]

    # Train the model with EarlyStopping callback
    model.fit(X_train_fold, Y_train_fold, 
              batch_size=10000, 
              epochs=100, 
              verbose=1, 
              callbacks=[EarlyStopping(min_delta=0.001, monitor="loss")]
              )

    # Evaluate the model on validation data
    valid_loss, valid_acc, valid_f1, valid_recall, valid_precision = model.evaluate(X_val_fold, Y_val_fold)
    valid_accs.append(valid_acc)
    valid_f1s.append(valid_f1)
    valid_recalls.append(valid_recall)
    valid_precisions.append(valid_precision)
    print("===================================")
    print("Validation accuracy:", valid_acc)
    print("Validation F1-score:", valid_f1)
    print("Validation recall:", valid_recall)
    print("Validation precision:", valid_precision)
print("###################################")
print("avg Validation accuracy:", np.mean(valid_accs))
print("avg Validation F1-score:", np.mean(valid_f1s))
print("avg Validation recall:", np.mean(valid_recalls))
print("avg Validation precision:", np.mean(valid_precisions))


Epoch 1/100
70/70 [==============================] - 1s 8ms/step - loss: 0.4808 - accuracy: 0.7906 - f1_score: 0.7906 - recall: 0.7906 - precision: 0.7906
Epoch 2/100
70/70 [==============================] - 1s 8ms/step - loss: 0.4608 - accuracy: 0.7980 - f1_score: 0.7980 - recall: 0.7980 - precision: 0.7980
Epoch 3/100
70/70 [==============================] - 1s 9ms/step - loss: 0.4586 - accuracy: 0.7985 - f1_score: 0.7985 - recall: 0.7985 - precision: 0.7985
Epoch 4/100
5459/5459 [==============================] - 2s 282us/step - loss: 0.4593 - accuracy: 0.7990 - f1_score: 0.7990 - recall: 0.7990 - precision: 0.7990
Validation accuracy: 0.7990073561668396
Validation F1-score: 0.7990077137947083
Validation recall: 0.7990078330039978
Validation precision: 0.7990078330039978
Epoch 1/100
70/70 [==============================] - 1s 10ms/step - loss: 0.4575 - accuracy: 0.7988 - f1_score: 0.7988 - recall: 0.7988 - precision: 0.7988
Epoch 2/100
5459/5459 [==============================] - 2s